In [106]:
from packaging.version import parse as version_parse
import requests
import json
from urllib.request import urlretrieve
import re
from urllib.error import HTTPError, URLError
from json import JSONDecodeError
from socket import gaierror
import tarfile
import shutil
from collections import defaultdict
import pprint


def get_json(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
        data = operUrl.read()
        jsonData = json.loads(data)
    else:
        print(f"Error receiving data for {url}")
    return jsonData
import requests 

def download_compressed(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [13]:
unoffical_list = urlretrieve("https://raw.githubusercontent.com/wiki/arduino/Arduino/Unofficial-list-of-3rd-party-boards-support-urls.md","Unofficial-list-of-3rd-party-boards-support-urls.md")
find_json_re = r"(https?:\/\/[a-zA-Z0-9-_./]+.json)"
find_name_re = r"package_.+_index.json"
json_links = []
for line in open('Unofficial-list-of-3rd-party-boards-support-urls.md'):
    search = re.search(find_json_re,line)
    if search:
        link = search.group(0)
        name = re.search(find_name_re,link)
        if name:
            links.append((name.group(0),link))
        else:
            #json links that don't conform
            print(link)


https://raw.githubusercontent.com/DFRobot/DFRobotDuinoBoard/master/package_dfrobot_iot_mainboard.json


## Download JSON files

In [15]:
jsons = []
for name,link in links:
#     print(link)
    try:
        jsons.append(get_json(link))
        with open("JSON/"+name, 'w') as f:
            json.dump(jsons[-1],f)
    except URLError:
        print('url:', link)
    except HTTPError:
        print("http:",link)
    except JSONDecodeError:
        print('json:', link)
    except gaierror:
        print('gai:', link)

url: http://www.arducam.com/downloads/ESP8266_UNO/package_ArduCAM_index.json
url: http://www.arducam.com/downloads/ESP32_UNO/package_ArduCAM_ESP32S_UNO_index.json
url: https://git.oschina.net/dfrobot/FireBeetle-ESP32/raw/master/package_esp32_index.json
url: http://rfduino.com/package_rfduino_index.json


## Extract URL info from JSON files

In [20]:
platforms = jsons[0]['packages'][0]['platforms']

arch_urls = {}
for j in jsons:
    for package in j['packages']:
        for i,p in enumerate(package['platforms']):
            arch = p['name'] +' - ' + p['architecture']
            if arch in arch_urls.keys():
                if version_parse(p['version']) > version_parse(arch_urls[arch]['version']):
                    arch_urls[arch] = {'version':p['version'],'url':p['url'],'name':p['url'][p['url'].rfind('/')+1:]}
                else:
                    continue
            else:
                arch_urls[arch] = {'version':p['version'],'url':p['url'],
                                   'name':p['url'][p['url'].rfind('/')+1:],
                                   'maintainer': package}
# arch_urls

## Download and extract all boards.txt

In [24]:
arch_urls['Adafruit AVR Boards - avr']

{'version': '1.4.13',
 'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-avr-1.4.13.tar.bz2',
 'name': 'adafruit-avr-1.4.13.tar.bz2'}

In [63]:
all_vids = []
import os

In [137]:
board_dict = defaultdict(list)
inv_dict = {}
vid_finder_re = r".vid.[0-9]+="
already_used_urls = []
num = 0
used_filenames = []
for k in arch_urls.keys():
    compressed_name = f'compressed/{arch_urls[k]["name"]}'
    if not (arch_urls[k]['url'] in already_used_urls):
        if not os.path.isfile(compressed_name):
            print(f"downloading to: {compressed_name}")
            download_compressed(arch_urls[k]['url'],compressed_name)
            already_used_urls = arch_urls[k]['url']
    else:
        continue
    print(compressed_name)
    
    # use /boards.txt to exclude annoying cases like ./._boards.txt - see Akafugu...
    if tarfile.is_tarfile(compressed_name):
        tf = tarfile.open(compressed_name)
        boards = [(tf.extractfile(info),info.name.replace('/','-')) for info in tf.getmembers() if ('/boards.txt' in info.name)]
    elif zipfile.is_zipfile(compressed_name):
        zp = zipfile.ZipFile(compressed_name)
        boards = [(zp.open(name),name.replace('/','-')) for name in zp.namelist() if ('/boards.txt' in name)]

    for f,filename in boards:
        if 'boards'+filename in used_filenames:
            # avoid name conflicts
            filename += "name_conflict"
        with open('boards/'+filename, 'wb') as boards_txt:
            iterable = iter(f.readlines())
            for line in iterable:
                boards_txt.write(line)

                line = line.decode().strip()
                if '.name=' in line:
                    name = line[line.index("=")+1:]
                    print(name)
                elif re.search(vid_finder_re,line):
                    num += 1
                    new_line = next(iterable)
                    boards_txt.write(new_line)
                    vid_pid = line[-4:] +":"+ new_line.decode().strip()[-4:]
                    inv_dict[name] = vid_pid
                    board_dict[vid_pid].append(name)
                    all_vids.append(vid_pid)

# clear duplicates
for k in board_dict.keys():
    board_dict[k] = list(set(board_dict[k]))

compressed/adafruit-avr-1.4.13.tar.bz2
Adafruit Flora
Adafruit Feather 32u4
Adafruit Feather 328P
Adafruit Gemma (ATtiny85 @ 8MHz)
Adafruit Trinket (ATtiny85 @ 8MHz)
Adafruit Trinket (ATtiny85 @ 16MHz)
Adafruit Metro
Pro Trinket 5V/16MHz (USB)
Pro Trinket 3V/12MHz (USB)
Pro Trinket 5V/16MHz (FTDI)
Pro Trinket 3V/12MHz (FTDI)
Adafruit Circuit Playground Classic
Adafruit ItsyBitsy 32u4 5V 16MHz
Adafruit ItsyBitsy 32u4 3V 8MHz
Adafruit Bluefruit Micro
Adafruit 32u4 Breakout
compressed/adafruit-samd-1.5.11.tar.bz2
Adafruit Feather M0
Adafruit Feather M0 Express
Adafruit M0 Radio (Native USB Port)
Adafruit Metro M0 Express
Adafruit Circuit Playground Express
Adafruit Gemma M0
Adafruit Trinket M0
Adafruit ItsyBitsy M0
Adafruit pIRkey
Adafruit Hallowing M0
Adafruit Crickit M0
Adafruit Metro M4 (SAMD51)
Adafruit Grand Central M4 (SAMD51)
Adafruit ItsyBitsy M4 (SAMD51)
Adafruit Feather M4 Express (SAMD51)
Adafruit Trellis M4 (SAMD51)
Adafruit PyPortal M4 (SAMD51)
Adafruit PyPortal M4 Titano (SA

In [141]:
with open('boards.py','w') as f:
    f.write('boards_dict = ')
    pprint.pprint(dict(board_dict),stream=f)
with open('boards.json', 'w') as f:
    f.write(json.dumps(board_dict))

In [142]:
with open('boards.json','r') as f:
    loaded_dict = json.loads(''.join(f.readlines()))

In [156]:
import boards
boards.boards_dict

{'239A:8004': ['Adafruit Flora'],
 '239A:800C': ['Adafruit Feather 32u4'],
 '239A:8011': ['Adafruit Circuit Playground Classic'],
 '239A:000E': ['Adafruit ItsyBitsy 32u4 5V 16MHz'],
 '239A:000D': ['Adafruit ItsyBitsy 32u4 3V 8MHz'],
 '239A:800A': ['Adafruit Bluefruit Micro'],
 '239A:8001': ['Adafruit 32u4 Breakout'],
 '239A:800B': ['Adafruit Feather M0'],
 '239A:000B': ['Adafruit Feather M0'],
 '239A:0015': ['Adafruit Feather M0'],
 '239A:801B': ['Adafruit Feather M0 Express'],
 '239A:001B': ['Adafruit Feather M0 Express'],
 '239A:8014': ['Adafruit M0 Radio (Native USB Port)'],
 '239A:0014': ['Adafruit M0 Radio (Native USB Port)'],
 '239A:8013': ['Adafruit Metro M0 Express'],
 '239A:0013': ['Adafruit Metro M0 Express'],
 '239A:8018': ['Adafruit Circuit Playground Express'],
 '239A:0019': ['Adafruit Circuit Playground Express'],
 '239A:801C': ['Adafruit Gemma M0'],
 '239A:001C': ['Adafruit Gemma M0'],
 '239A:801E': ['Adafruit Trinket M0', 'Adafruit pIRkey'],
 '239A:001E': ['Adafruit Tri